# Создание базы данных

In [1]:
import sqlite3

In [21]:
# create_queries = ['''
# CREATE TABLE "texts" ( 
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 
# '''
# CREATE TABLE "sents" (
# 	"id"	INTEGER,
# 	"text_id"	INTEGER,
# 	"sent"	TEXT,
# 	"lemmatized"	TEXT,
# 	"pos_in_text"	INTEGER,
# 	FOREIGN KEY("text_id") REFERENCES "texts"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );''', 

# '''
# CREATE TABLE "govern_models" (
# 	"id"	INTEGER,
# 	"model"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_imgs" (
# 	"id"	INTEGER,
# 	"math_tagname"	TEXT,
# 	"filename"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "math_tags" (
# 	"id"	INTEGER,
# 	"feature"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "lemmas" (
# 	"id"	INTEGER,
# 	"name"	TEXT UNIQUE,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "pos" (
# 	"id"	INTEGER,
# 	"name"	TEXT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "deprels" (
# 	"id"	INTEGER,
# 	"name"	INTEGER,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "tokens" (
# 	"id"	INTEGER,
# 	"sent_id"	INTEGER,
# 	"word_in_sent"	INTEGER,
# 	"token"	TEXT,
# 	FOREIGN KEY("sent_id") REFERENCES "sents"("id") ON DELETE RESTRICT,
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''', 

# '''
# CREATE TABLE "math_annotation" (
# 	"id"	INTEGER,
# 	"id_token_start"	INTEGER,
# 	"id_token_end"	INTEGER,
# 	"feature_id"	INTEGER,
# 	"govern_model_id"	INTEGER,
# 	FOREIGN KEY("feature_id") REFERENCES "math_tags"("id"),
# 	FOREIGN KEY("govern_model_id") REFERENCES "govern_models"("id"),
# 	FOREIGN KEY("id_token_start") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("id_token_end") REFERENCES "tokens"("id"),
# 	PRIMARY KEY("id" AUTOINCREMENT)
# );
# ''',

# '''
# CREATE TABLE "grammar_annotation" (
# 	"token_id"	INTEGER,
# 	"lemma_id"	INTEGER,
# 	"pos_id"	INTEGER,
# 	"deprel_id"	INTEGER,
# 	"head_id"	INTEGER,
# 	FOREIGN KEY("lemma_id") REFERENCES "lemmas"("id"),
# 	FOREIGN KEY("token_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("deprel_id") REFERENCES "deprels"("id"),
# 	FOREIGN KEY("head_id") REFERENCES "tokens"("id"),
# 	FOREIGN KEY("pos_id") REFERENCES "pos"("id"),
# 	PRIMARY KEY("token_id")
# );
# '''
#                  ]

In [23]:
# db_path = 'math_corpus_database.db'
# conn = sqlite3.connect(db_path, check_same_thread=False)
# cur = conn.cursor()

In [103]:
# for q in create_queries:
#     cur.execute(q)
#     conn.commit()
# conn.close()

# Парсинг и заполнение БД

In [2]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download ru_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 15.3/15.3 MB 7.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [4]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [5]:
db_path = 'math_corpus_database.db'

In [6]:
class DBHandler:
    conn = None
    cur = None

    def __init__(self, db_path):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.cur = self.conn.cursor()

    def __del__(self):
        self.conn.close()

    def add_text(self, text_name):
        
        self.cur.execute('''INSERT or IGNORE INTO texts (name)
                            VALUES (?)
                            RETURNING id''', (text_name,))
    
        added_id = self.cur.fetchone()
        self.conn.commit()
        
        if not added_id:
            self.cur.execute('''SELECT id
                                FROM texts
                                WHERE name = (?)''', (text_name, ))
            added_id = self.cur.fetchone()
        return added_id[0]
        
    def add_sent(self, text_id, sent, lemmatized, sent_count):
        
        self.cur.execute('''INSERT INTO sents (text_id, sent, lemmatized, pos_in_text)
                            VALUES (?, ?, ?, ?)
                            RETURNING id''', (text_id, sent, lemmatized, sent_count))
        
        added_id = self.cur.fetchone()
        self.conn.commit()
        return added_id[0]
    
    def add_tokens(self, tokens_info): 
        ":param tokens_info: list of tuples (sent_id, word_in_sent, token, char_start, char_end)"
        
        self.cur.executemany('''INSERT INTO tokens (sent_id, word_in_sent, token, char_start, char_end)
                                VALUES (?, ?, ?, ?, ?)''', tokens_info)
        
        self.conn.commit()
        
        self.cur.execute('''SELECT id 
                            FROM tokens
                            WHERE sent_id = (?)
                            ORDER BY word_in_sent''', (tokens_info[0][0],))
        added_ids = self.cur.fetchall()
        return [token_id[0] for token_id in added_ids]
    
    
    def add_lemmas(self, lemmas):
        lemmas = [(el,) for el in lemmas]
        self.cur.executemany('''INSERT or IGNORE
                            INTO lemmas (name)
                            VALUES (?)''', lemmas)
        self.conn.commit()
        
        lemma_ids = []
        for lemma in lemmas:
            self.cur.execute('''SELECT id
                                FROM lemmas
                                WHERE name = ?''', lemma)
            lemma_ids.append(self.cur.fetchone()[0])
        return lemma_ids
    
    
    def add_grammar_annot(self, token_grammar_info):
        ":param token_grammar_info: [(token_id, pos, deprel, head_id, lemma)]"
        
        self.cur.executemany('''INSERT INTO grammar_annotation (token_id, lemma_id, pos_id, deprel_id, head_id)
                                SELECT ?, 
                                        lemmas.id, 
                                        (SELECT pos.id FROM pos WHERE pos.name = ?),
                                        (SELECT deprels.id FROM deprels WHERE deprels.name = ?),
                                        ?
                                FROM lemmas
                                WHERE lemmas.name = ?''', token_grammar_info)
        self.conn.commit()
        
    def add_math_tags(self, tags):
        tags = [(el,) for el in tags]
        self.cur.executemany('''INSERT or IGNORE
                                INTO math_tags (feature)
                                VALUES (?)''', tags)
        self.conn.commit()
        
    def add_govern_models(self, models):
        models = [(el,) for el in models]
        self.cur.executemany('''INSERT or IGNORE
                                INTO govern_models (model)
                                VALUES (?)''', models)
        self.conn.commit()
        
    def get_sent_id(self, sentence):
        self.cur.execute('''SELECT id
                            FROM sents
                            WHERE sents.sent = (?)''', (sentence,))
        return self.cur.fetchone()[0]
    
    
    def add_math_annotation(self, annotation):
        """:param annotation: [{
            'sentence': str, 
            'segment_text': str,
            'char_start': int, 
            'char_end': int,
            'annot': str, 
            'govern_model': str}
            }]"""
        
        annot = list(set([an['annot'] for an in annotation]))
        self.add_math_tags(annot)
        govern_models = list(set([an['govern_model'] for an in annotation if an['govern_model']]))
        self.add_govern_models(govern_models)
        
        sent_ids = {}
        for annot in annotation:
            if not sent_ids.get(annot['sentence']):
                sent_ids[annot['sentence']] = self.get_sent_id(annot['sentence'])
            annot['sentence_id'] = sent_ids[annot['sentence']]
            
        self.cur.executemany('''INSERT INTO math_annotation (feature_id, govern_model_id, id_token_start, id_token_end)
                                SELECT math_tags.id, 
                                (SELECT govern_models.id FROM govern_models WHERE govern_models.model = :govern_model),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent 
                                LIMIT 1),
                                (SELECT tokens.id FROM tokens 
                                WHERE tokens.char_start >= :char_start AND tokens.char_end <= :char_end AND tokens.sent_id = :sentence_id
                                ORDER BY tokens.word_in_sent DESC
                                LIMIT 1)
                            FROM math_tags
                            WHERE math_tags.feature = :annot''', annotation)
        self.conn.commit()
        
        
db = DBHandler(db_path)

In [7]:
xml_filename = 'test.xml'

In [8]:
from bs4 import BeautifulSoup

In [9]:
with open(xml_filename, 'r', encoding='utf-8') as f:
    xml_file = f.read()
bs_data = BeautifulSoup(xml_file, "xml")
segs = bs_data.find_all('segment')

In [47]:
def parse_sentence(analysed_sent):
    parsed = {
        'tokens': [], 
        'lemmas': [], 
        'poses': [], 
        'deprels': [], 
        'head_id_in_sent': [], 
        'full_sentence': analysed_sent.text,
    }
    first_word_in_sent = 0
    
    for t in analysed_sent:
        parsed['tokens'].append(t.text)
        parsed['lemmas'].append(t.lemma_)
        parsed['poses'].append(t.pos_)
        parsed['deprels'].append(t.dep_.lower())
        
        if t.is_sent_start:
            first_word_in_sent = t.i
        
        parsed['head_id_in_sent'].append(t.head.i - first_word_in_sent)
    parsed['lemmatized_sent'] = ' '.join(parsed['lemmas'])
    return parsed

In [48]:
import re

In [49]:
def accum_token_info(parsed, sent_id):
    n_tokens = len(parsed['tokens'])
    token_info = []
    sent_string = parsed['full_sentence']
    prev_chars_count = 0
    for i in range(n_tokens):
        
        token = parsed['tokens'][i]
        m = re.search(re.escape(token), sent_string)
        char_start, char_end = m.start(), m.end()
        
        token_info.append((sent_id, i, parsed['tokens'][i], prev_chars_count + char_start, prev_chars_count + char_end))
        prev_chars_count += char_end
        sent_string = sent_string[char_end:]  # обрезаем строку, чтобы короткие слова не искались в начале строки
    return token_info

In [50]:
def accum_grammar_info(parsed, added_tokens):
    n_tokens = len(parsed['tokens'])
    head_ids = [added_tokens[hi] for hi in parsed['head_id_in_sent']]
    token_grammar_info = [
    (
        added_tokens[i], 
        parsed['poses'][i],
        parsed['deprels'][i], 
        head_ids[i],
        parsed['lemmas'][i]
    ) for i in range(n_tokens)
    ]
    return token_grammar_info

In [51]:
text_id = 1
sent_count = 0
annotation = []
segment_info= {}
for seg in segs:
    parent_id = seg.get('parent')
    if not parent_id:
        text = seg.get_text()
        if segment_info:
            annotation.append(segment_info)
        segment_info = {
            'text': text, 
            'segments_annot': []
        }
#         sentences = list(nlp(text).sents)
#         for sent in sentences:
#             sent_count += 1
#             parsed_sentence = parse_sentence(sent)
            
#             lemmas = parsed_sentence['lemmas']
#             added_lemmas = db.add_lemmas(lemmas)
            
#             sent_text = parsed_sentence['full_sentence']
#             sent_lemmatized = parsed_sentence['lemmatized_sent']
#             sent_id = db.add_sent(text_id, sent_text, sent_lemmatized, sent_count)
            
#             tokens_info = accum_token_info(parsed_sentence, sent_id)
#             added_tokens = db.add_tokens(tokens_info)
    
            
#             grammar_info = accum_grammar_info(parsed_sentence, added_tokens)
#             db.add_grammar_annot(grammar_info)
    else:
        if int(parent_id) == int(seg.get('id')) - 1:
            continue
        else:
            segment_text = seg.get_text()
            segment_info['segments_annot'].append((segment_text, seg.get('features'), seg.get('comment')))

In [52]:
annotation

[{'text': 'Степень с натуральным показателем.',
  'segments_annot': [('Степень',
    'math_tags;oper;power',
    'Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG'),
   ('натуральным', 'math_tags;term;set;nat', None),
   ('показателем', 'math_tags;visual;superscript', None)]},
 {'text': 'Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное. Считается, что а в первой степени равняется а.',
  'segments_annot': [('Определение', 'math_tags;comment;definition', None),
   ('степени',
    'math_tags;oper;power',
    'Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG'),
   ('а', 'math_tags;term;var;common;letter;latin;a', None),
   ('энной', 'math_tags;term;var;common;letter;latin;n', None),
   ('степени',
    'math_tags;oper;power',
    'Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG'),
   ('равняется', 'math_tags;pred;equal', None),
   ('произведению', 'math_tag

In [53]:
def find_segment(annotation_piece):
    text, segments = annotation_piece['text'], annotation_piece['segments_annot']
    
    sentences = list(nlp(text).sents)
    sentences_starts = [s.start_char for s in sentences]
    prev_chars_count = 0
    segments_info = []
    for seg in segments:
        m = re.search('(?<![A-я])' + re.escape(seg[0]) + '(?![A-я])', text)  # чтобы не находило в середине слова
        local_start, local_end = m.start(), m.end()  # начало и конец совпадения в обрезанной строке
        text = text[local_end:]
        global_start, global_end = local_start + prev_chars_count, local_end + prev_chars_count
        segment_sent = sentences_starts.index([s for s in sentences_starts if s <= global_start][-1])
        prev_chars_count = global_end
        segments_info.append({'sentence': sentences[segment_sent].text, 
                              'segment_text': seg[0],
                              'char_start': global_start - sentences_starts[segment_sent], 
                              'char_end': global_end - sentences_starts[segment_sent],
                              'annot': seg[1], 
                            'govern_model': seg[2]})
    return segments_info

In [54]:
all_math_annotation = []
for annot in annotation:
    all_math_annotation.extend(find_segment(annot))

In [55]:
all_math_annotation

[{'sentence': 'Степень с натуральным показателем.',
  'segment_text': 'Степень',
  'char_start': 0,
  'char_end': 7,
  'annot': 'math_tags;oper;power',
  'govern_model': 'Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG'},
 {'sentence': 'Степень с натуральным показателем.',
  'segment_text': 'натуральным',
  'char_start': 10,
  'char_end': 21,
  'annot': 'math_tags;term;set;nat',
  'govern_model': None},
 {'sentence': 'Степень с натуральным показателем.',
  'segment_text': 'показателем',
  'char_start': 22,
  'char_end': 33,
  'annot': 'math_tags;visual;superscript',
  'govern_model': None},
 {'sentence': 'Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное.',
  'segment_text': 'Определение',
  'char_start': 0,
  'char_end': 11,
  'annot': 'math_tags;comment;definition',
  'govern_model': None},
 {'sentence': 'Определение степени: а в энной степени равняется произведению энн множителей, ка

In [105]:
# db.add_math_annotation(all_math_annotation)

In [13]:
segs

[<segment features="math_tags" filename="" id="1" state="active" subcorp=""><segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment> с <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment> <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>.</segment>,
 <segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment>,
 <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment>,
 <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>,
 <segment feat

In [133]:
t = 'Показатель делимого, из показателя делимого вычитают показатель делителя.Например: два в пятой степени разделим на два в третьей степени, вычитаем показатели, получаем два во второй степени и четыре.'

In [41]:
test_text = '''<?xml version='1.0' encoding='utf-8'?>
<document>
<header>
<textfile>Texts/limit_test.txt</textfile>
<lang>english</lang></header><body>
<segment id='45' features='math_tags' state='active' subcorp='' filename=''>И дальше мы говорим, что вот какое-то число <segment id='164' features='math_tags;term' state='active' subcorp='' filename=''><segment id='163' features='math_tags;term;var' state='active' subcorp='' filename=''>а</segment> большое</segment> является <segment id='165' features='math_tags;function' state='active' subcorp='' filename=''><segment id='166' features='math_tags;term' state='active' subcorp='' filename=''>пределом </segment><segment id='167' features='math_tags;visual' state='active' subcorp='' filename=''>при <segment id='168' features='math_tags;term' state='active' subcorp='' filename=''>икс </segment></segment><segment id='169' features='math_tags;visual' state='active' subcorp='' filename=''>стремящемся к <segment id='170' features='math_tags;term' state='active' subcorp='' filename=''><segment id='171' features='math_tags;term;var' state='active' subcorp='' filename=''>а</segment> маленькому</segment></segment></segment>, если, какое бы малое <segment id='173' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment> вы вот тут не взяли… Да, потому что <segment id='174' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment>, оно про <segment id='175' features='math_tags;term' state='active' subcorp='' filename=''>а большое</segment>.</segment>
<segment id='46' features='math_tags' state='active' subcorp='' filename=''>Какое бы маленькое <segment id='172' features='math_tags;term' state='active' subcorp='' filename=''>эпсилон</segment> будут не взяли, существует такое <segment id='176' features='math_tags;term' state='active' subcorp='' filename=''>дельта</segment>, что все <segment id='177' features='math_tags;term' state='active' subcorp='' filename=''>иксы</segment>, которые лежат вот здесь, попадают вот сюда.</segment>
</body>
</document>'''

In [42]:
bs_data2 = BeautifulSoup(test_text, "xml")

In [43]:
sent_segments = bs_data2.find('body').find_all('segment', recursive=False)

In [44]:
bs_data2.find('body').find_all('segment', recursive=True)

[<segment features="math_tags" filename="" id="45" state="active" subcorp="">И дальше мы говорим, что вот какое-то число <segment features="math_tags;term" filename="" id="164" state="active" subcorp=""><segment features="math_tags;term;var" filename="" id="163" state="active" subcorp="">а</segment> большое</segment> является <segment features="math_tags;function" filename="" id="165" state="active" subcorp=""><segment features="math_tags;term" filename="" id="166" state="active" subcorp="">пределом </segment><segment features="math_tags;visual" filename="" id="167" state="active" subcorp="">при <segment features="math_tags;term" filename="" id="168" state="active" subcorp="">икс </segment></segment><segment features="math_tags;visual" filename="" id="169" state="active" subcorp="">стремящемся к <segment features="math_tags;term" filename="" id="170" state="active" subcorp=""><segment features="math_tags;term;var" filename="" id="171" state="active" subcorp="">а</segment> маленькому</s

In [46]:
import re

In [60]:
from collections import defaultdict

In [61]:
text_segments = defaultdict(list)
first_layer = []
for i, seg in enumerate(bs_data2.find('body').find_all('segment', recursive=True)):
    print(i, seg.get_text())
    if seg.parent.name == 'body':
        text_segment = seg.get_text()
        #text_segments.append(seg.get_text())
    elif seg.parent.get('features') == 'math_tags':
        #first_layer.append(seg)
        text_segments[text_segment].append(seg)
        
print()
print(text_segments)
# print(first_layer)

0 И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли… Да, потому что эпсилон, оно про а большое.
1 а большое
2 а
3 пределом при икс стремящемся к а маленькому
4 пределом 
5 при икс 
6 икс 
7 стремящемся к а маленькому
8 а маленькому
9 а
10 эпсилон
11 эпсилон
12 а большое
13 Какое бы маленькое эпсилон будут не взяли, существует такое дельта, что все иксы, которые лежат вот здесь, попадают вот сюда.
14 эпсилон
15 дельта
16 иксы

defaultdict(<class 'list'>, {'И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли… Да, потому что эпсилон, оно про а большое.': [<segment features="math_tags;term" filename="" id="164" state="active" subcorp=""><segment features="math_tags;term;var" filename="" id="163" state="active" subcorp="">а</segment> большое</segment>, <segment features="math_tags;f

In [152]:
result = {}

for text, fl_segs in text_segments.items():
#     print(text)
    sentences = list(nlp(text).sents)
    sentences_starts = [s.start_char for s in sentences]
#     print(sentences_starts)
#     print()
#     print()
    text_area = text
    prev_char_counter = 0
    for fl_seg in fl_segs: 
#         print(fl_seg.get_text())
#         print('text_area: ', text_area)
        m = re.search('(?<![A-я])' + re.escape(fl_seg.get_text()) + '(?![A-я])', text_area)
        segment_sent = sentences_starts.index([s for s in sentences_starts if s <= prev_char_counter + m.start()][-1])
#         print('segment_sent=', segment_sent)
        text_area = text_area[m.end():]
        
#         print(prev_char_counter + m.start(), prev_char_counter + m.end())
#         print(text[prev_char_counter + m.start(): prev_char_counter + m.end()])
        span_in_sentence = prev_char_counter + m.start() - sentences_starts[segment_sent], prev_char_counter + m.end() - sentences_starts[segment_sent]
#         print('span in sentence=', span_in_sentence)
#         print('seg_in_sent', sentences[segment_sent][span_in_sentence[0]: span_in_sentence[1]])
        prev_char_counter += m.end()
        result[fl_seg.get_text()] = {
            'sent': sentences[segment_sent].text,
            'char_start': span_in_sentence[0],
            'char_end': span_in_sentence[1],
            'childrens': fl_seg
        }
#         print()
print(result)

{'а большое': {'sent': 'Да, потому что эпсилон, оно про а большое.', 'char_start': 32, 'char_end': 41, 'childrens': <segment features="math_tags;term" filename="" id="175" state="active" subcorp="">а большое</segment>}, 'пределом при икс стремящемся к а маленькому': {'sent': 'И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…', 'char_start': 63, 'char_end': 106, 'childrens': <segment features="math_tags;function" filename="" id="165" state="active" subcorp=""><segment features="math_tags;term" filename="" id="166" state="active" subcorp="">пределом </segment><segment features="math_tags;visual" filename="" id="167" state="active" subcorp="">при <segment features="math_tags;term" filename="" id="168" state="active" subcorp="">икс </segment></segment><segment features="math_tags;visual" filename="" id="169" state="active" subcorp="">стремящемся к <segment features="math_tags;term" f

In [153]:
prev_parent = 'пределом при икс стремящемся к а маленькому'
search_area = prev_parent
search_area_char_count = 0
for s in result['пределом при икс стремящемся к а маленькому']['childrens'].find_all('segment'):
    current_parent = s.parent.get_text()
    if prev_parent != current_parent:
        search_area = current_parent
        search_area_char_count = 0
    print(s.get_text())
    print('search_area: ', search_area)
    m = re.search('(?<![A-я])' + re.escape(s.get_text().strip()) + '(?![A-я])', search_area)
    print('start_end:', search_area_char_count + m.start(), search_area_char_count + m.end())

    parent_start_end = result[current_parent]['char_start'], result[current_parent]['char_end']
    print('parent_start_end:', parent_start_end)
    print('sentence_start_end', parent_start_end[0] + search_area_char_count + m.start(), parent_start_end[0] + search_area_char_count + m.end())
    print()
    result[s.get_text()] = {
        'sent': result[current_parent]['sent'],
        'char_start': parent_start_end[0] + search_area_char_count + m.start(),
        'char_end': parent_start_end[0] + search_area_char_count + m.end(),
        'childrens': s
    }
    
    search_area = search_area[m.end():]
    search_area_char_count += m.end()
    prev_parent = current_parent

пределом 
search_area:  пределом при икс стремящемся к а маленькому
start_end: 0 8
parent_start_end: (63, 106)
sentence_start_end 63 71

при икс 
search_area:   при икс стремящемся к а маленькому
start_end: 9 16
parent_start_end: (63, 106)
sentence_start_end 72 79

икс 
search_area:  при икс 
start_end: 4 7
parent_start_end: (72, 79)
sentence_start_end 76 79

стремящемся к а маленькому
search_area:  пределом при икс стремящемся к а маленькому
start_end: 17 43
parent_start_end: (63, 106)
sentence_start_end 80 106

а маленькому
search_area:  стремящемся к а маленькому
start_end: 14 26
parent_start_end: (80, 106)
sentence_start_end 94 106

а
search_area:  а маленькому
start_end: 0 1
parent_start_end: (94, 106)
sentence_start_end 94 95



In [155]:
for seg in result.keys():
    start, end = result[seg]['char_start'], result[seg]['char_end']
    sent = result[seg]['sent']
    print(seg)
    print(sent[:start] + '\033[1m' + sent[start:end] + '\033[0m' + sent[end:])
    print()

а большое
Да, потому что эпсилон, оно про а большое.

пределом при икс стремящемся к а маленькому
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

эпсилон
Какое бы маленькое эпсилон будут не взяли, существует такое дельта, что все иксы, которые лежат вот здесь, попадают вот сюда.

дельта
Какое бы маленькое эпсилон будут не взяли, существует такое дельта, что все иксы, которые лежат вот здесь, попадают вот сюда.

иксы
Какое бы маленькое эпсилон будут не взяли, существует такое дельта, что все иксы, которые лежат вот здесь, попадают вот сюда.

пределом 
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

при икс 
И дальше мы говорим, что вот какое-то число а большое является пределом при икс стремящемся к а маленькому, если, какое бы малое эпсилон вы вот тут не взяли…

икс 


In [13]:
for s in sent_segments[54].find_all('segment', recursive=False):
    print(s.text)

существует
предел при эн стремящимся к бесконечности икс эн
равен а


In [90]:
for s in sent_segments:
    for seg in s.find_all('segment'):
        if not seg.parent.get('features') == 'math_tags':
            print(seg.text, '\t', seg.parent.text)
            print()

стремящимся к бесконечности 	 предел при эн стремящимся к бесконечности икс эн

икс эн 	 предел при эн стремящимся к бесконечности икс эн

икс 	 икс эн

эн 	 икс эн



In [ ]:
sentences = list(nlp(text).sents)
sentences_starts = [s.start_char for s in sentences]
prev_chars_count = 0
segments_info = []
for seg in segments:
    m = re.search('(?<![A-я])' + re.escape(seg[0]) + '(?![A-я])', text)  # чтобы не находило в середине слова
    local_start, local_end = m.start(), m.end()  # начало и конец совпадения в обрезанной строке
    text = text[local_end:]
    global_start, global_end = local_start + prev_chars_count, local_end + prev_chars_count
    segment_sent = sentences_starts.index([s for s in sentences_starts if s <= global_start][-1])
    prev_chars_count = global_end
    segments_info.append({'sentence': sentences[segment_sent].text, 
                          'segment_text': seg[0],
                          'char_start': global_start - sentences_starts[segment_sent], 
                          'char_end': global_end - sentences_starts[segment_sent],
                          'annot': seg[1], 
                        'govern_model': seg[2]})

In [ ]:
segment_info = {'sentence': '',
               'segment_text': '', 
               'char_start_in_sent': 0, 
               'char_end_in_sent': 0}

for sent_segment in sent_segments:
    first_layer = []
    spacy_sentences = list(nlp(sent_segment.get_text()).sents)
    sentences_starts = [s.start_char for s in spacy_sentences]
    text = sent.segment.get_text()
    prev_chars_count = 0
    for first_layer_seg in sent_segment.find_all('segment', recursive=False):
        seg_char_span = get_char_span_in_sent(first_layer_seg.get_text(), text, prev_chars_count)
        segment_sent = sentences_starts.index([s for s in sentences_starts if s <= seg_char_span[0][0]][-1])
        text = text[seg_char_span[1][1]:]
        prev_char_count = seg_char_span[0][1]
        first_layer = []
        

In [ ]:
def

если в сегменте предложения одно предложение:
- для каждого сегмента первого уровня sentence = 0
если в сегменте предложения несколько предложений:
- для каждого сегмента первого уровня находим sentence_id, parent_gloabl_start

In [ ]:
def get_char_span_in_sent(seg_text, search_area, search_area_char_start):
    m = re.search('(?<![A-я])' + re.escape(seg_text) + '(?![A-я])', search_area)  # чтобы не находило в середине слова
    start_within_search_area, end_within_search_area = m.start(), m.end()
    start_in_sent, end_in_sent = start_within_search_area + search_area_char_start, end_within_search_area + search_area_char_start
    return (start_in_sent, end_in_sent), (start_within_search_area, end_within_search_area)

In [41]:
annotation = []
for sent_segment in sent_segments:
    sent_seg_text = sent_segment.get_text()
    spacy_sentences = nlp(sent_seg_text).sents
    print(list(spacy_sentences))
    for sent in spacy_sentences:
        print(sent.text)

[Степень с натуральным показателем.]
[Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное., Считается, что а в первой степени равняется а.]
[Например, три в третьей степени - это произведение трех троек, равно такое произведение числу двадцать семь.]
[Минус пять во второй степени: для того, чтобы вычислить нужно минус пять умножить на минус пять, получим двадцать пять, семь в первой степени равно семи.]
[Стоит записать еще несколько свойств, например: ноль в энной степени равен нулю, единица в энной степени равна единице для любого натурального числа энн, ноль в нулевой степени - не определено, такое выражение не определено.]
[Свойства степеней.]
[Первое свойство: а в эммой степени умноженное на а в энной степени равняется а в степени эмм плюс энн.]
[Проиллюстрируем это свойство на примере: два в пятой степени умножим на два в третьей степени, значит свойство это говорит о следующем: при умножении степеней

In [22]:
bs_data.find('body').get_text()

'\nСтепень с натуральным показателем. Определение степени: а в энной степени равняется произведению энн множителей, каждый из которых равен а, где энн - число натуральное. Считается, что а в первой степени равняется а. Например, три в третьей степени - это произведение трех троек, равно такое произведение числу двадцать семь. Минус пять во второй степени: для того, чтобы вычислить нужно минус пять умножить на минус пять, получим двадцать пять, семь в первой степени равно семи. Стоит записать еще несколько свойств, например: ноль в энной степени равен нулю, единица в энной степени равна единице для любого натурального числа энн, ноль в нулевой степени - не определено, такое выражение не определено. Свойства степеней. Первое свойство: а в эммой степени умноженное на а в энной степени равняется а в степени эмм плюс энн. Проиллюстрируем это свойство на примере: два в пятой степени умножим на два в третьей степени, значит свойство это говорит о следующем: при умножении степеней с одинаковым

In [27]:
import re

In [31]:
re.search(r'Texts\/(.*?)\.txt', bs_data.find('textfile').get_text()).group(1)

'Степень с натуральным показателем'

In [73]:
sentences[0].find_all('segment')

[<segment comment="Модель управления: X в Y.ORD.LOC.SG степени; X в степени Y.NOM.SG" features="math_tags;oper;power" filename="" id="110" parent="1" state="active" subcorp="">Степень</segment>,
 <segment features="math_tags;term;set;nat" filename="" id="113" parent="1" state="active" subcorp="">натуральным</segment>,
 <segment features="math_tags;visual;superscript" filename="" id="114" parent="1" state="active" subcorp="">показателем</segment>]

In [ ]:
def 

In [ ]:
class AnnotationParser:
    
    def __init__(self, xml_file, database_path, textname=None):
        
        with open(xml_filename, 'r', encoding='utf-8') as f:
            xml_file = f.read()
        self.bs_data = BeautifulSoup(xml_file, "xml")
        self.db = DBHandler(db_path)
        
        if not textname:
            textname = re.search(r'Texts\/(.*?)\.txt', 
                                 re.bs_data.find('textfile').get_text()).group(1)
        self.text_id = self.db.add_text(textname)
        
        
    @staticmethod
    def parse_sentence_info_(analysed_sent):
        parsed = {
            'tokens': [], 
            'lemmas': [], 
            'poses': [], 
            'deprels': [], 
            'head_id_in_sent': [], 
            'full_sentence': analysed_sent.text,
        }
        first_word_in_sent = 0

        for t in analysed_sent:
            parsed['tokens'].append(t.text)
            parsed['lemmas'].append(t.lemma_)
            parsed['poses'].append(t.pos_)
            parsed['deprels'].append(t.dep_.lower())

            if t.is_sent_start:
                first_word_in_sent = t.i

            parsed['head_id_in_sent'].append(t.head.i - first_word_in_sent)
        parsed['lemmatized_sent'] = ' '.join(parsed['lemmas'])
        return parsed
    
    @staticmethod
    def accum_token_info_(parsed, sent_id):
        n_tokens = len(parsed['tokens'])
        token_info = []
        sent_string = parsed['full_sentence']
        prev_chars_count = 0
        for i in range(n_tokens):
            token = parsed['tokens'][i]
            m = re.search(re.escape(token), sent_string)
            char_start, char_end = m.start(), m.end()

            token_info.append((sent_id, i, parsed['tokens'][i], prev_chars_count + char_start, prev_chars_count + char_end))
            prev_chars_count += char_end
            sent_string = sent_string[char_end:]  # обрезаем строку, чтобы короткие слова не искались в начале строки
        return token_info
    
    @staticmethod
    def accum_grammar_info_(parsed, added_tokens):
        n_tokens = len(parsed['tokens'])
        head_ids = [added_tokens[hi] for hi in parsed['head_id_in_sent']]
        token_grammar_info = [
        (
            added_tokens[i], 
            parsed['poses'][i],
            parsed['deprels'][i], 
            head_ids[i],
            parsed['lemmas'][i]
        ) for i in range(n_tokens)
        ]
        return token_grammar_info
        
    def parse_sentences(self):
        full_text = self.bs_data.find('body').get_text()
        sent_count = 0
        for sent in nlp(full_text).sents:
            sent_count += 1
            parsed_sentence = AnnotationParser.parse_sentence_info_(sent)
            
            lemmas = parsed_sentence['lemmas']
            added_lemmas = self.db.add_lemmas(lemmas)
            
            sent_text = parsed_sentence['full_sentence']
            sent_lemmatized = parsed_sentence['lemmatized_sent']
            sent_id = self.db.add_sent(text_id, sent_text, sent_lemmatized, sent_count)
            
            tokens_info = AnnotationParser.accum_token_info_(parsed_sentence, sent_id)
            added_tokens = self.db.add_tokens(tokens_info)
            
            grammar_info = AnnotationParser.accum_grammar_info_(parsed_sentence, added_tokens)
            self.db.add_grammar_annot(grammar_info)
            
    def parse_math_annotation(self):
        